In [1]:
import autogen
import pandas as pd
from time import time
from datetime import datetime
import json

flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.


In [2]:
config_list_questions = autogen.config_list_from_json("model_config_questions.json")
config_list_questions

[{'model': 'tinyllama:1.1b-chat-v1-q2_K',
  'base_url': 'http://127.0.0.1:11434/v1',
  'api_key': 'ollama',
  'price': [0, 0],
  'max_tokens': 1000}]

In [3]:
config_list_answers = autogen.config_list_from_json("model_config_answers.json")
config_list_answers

[{'model': 'tinyllama:chat',
  'base_url': 'http://127.0.0.1:11434/v1',
  'api_key': 'ollama',
  'price': [0, 0],
  'max_tokens': 1000}]

In [4]:
agent_questions = autogen.AssistantAgent(
    name = "Agent Questions",
    system_message="Your task is to ask a question about computer science.",
    llm_config = {"config_list": config_list_questions},
    code_execution_config = False,
    human_input_mode = "NEVER",
    function_map = None
)
agent_questions

In [5]:
agent_answers = autogen.UserProxyAgent(
    name = "Agent Answers",
    system_message="Your task is to answer a question about computer science and ask for another question.",
    llm_config = {"config_list": config_list_answers},
    code_execution_config = False,
    human_input_mode = "NEVER",
    function_map = None
)
agent_answers

In [6]:
chat_results = agent_questions.initiate_chat(agent_answers, message="What is a linked-list in computer science?", max_turns=3)

Agent Questions (to Agent Answers):

What is a linked-list in computer science?

--------------------------------------------------------------------------------
Agent Answers (to Agent Questions):

In computer science, a linked-list (also known as an adjacency list or neighbor list) is a data structure that holds a collection of data nodes, with each node being associated to one or more adjacent pairs of nodes with the same key. The key is used to identify unique subgroups of nodes in a linked list, referred to as sub-subtrees or neighbor lists, which are created after each node has been added to the list. 

A linked-list can be thought of as a nested tree structure where nodes are connected by links to their neighbors, with each node having one or more predecessors and successors, which represent the neighbors who preceded it in its subtree. In other words, a linked-list represents an adjacency matrix in linear algebra.

In computer science, linked-lists are often used as data contai

---

#### Save Results

In [7]:
timestamp = datetime.fromtimestamp(time()).strftime("%Y-%m-%d_%H-%M-%S")
timestamp

'2024-11-07_13-37-57'

In [8]:
df_chat_history = pd.DataFrame.from_records(chat_results.chat_history)
df_chat_history

,content,role,name
0,What is a linked-list in computer science?,assistant,Agent Questions
1,"In computer science, a linked-list (also known...",user,Agent Answers
2,Sure! A linked-list is a collection structure ...,assistant,Agent Questions
3,I did not understand the last part in your pre...,user,Agent Answers
4,Certainly! Linked-lists are a type of data str...,assistant,Agent Questions
5,Thank you for clarifying that for me. I hope t...,user,Agent Answers


In [9]:
df_chat_history.to_excel(f"chat_results/chat_history_{timestamp}.xlsx", index=False)

In [10]:
with open(f"chat_results/chat_cost_{timestamp}.json", "w") as json_file:
    json.dump(chat_results.cost, json_file)